In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from pprint import pprint
import pandas as pd
import numpy as np


# Scraping Data from Twitter
The following block will use Selenium to scrape tweets from twitter

In [5]:
def infinity_scroll(driver,filename):
    try:
        for i in range(1000):
            
            soupObj = BeautifulSoup(driver.page_source,'html.parser')
            
            with open(filename,'a') as f: #remember to delete the file everytime you run it.
                f.write(str(soupObj))
            
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(2)

    except:
        return
        
    return


def scrape_tweets(twitter_handle,from_date,to_date,filename):
    
    url = f'https://twitter.com/search?q=(from%3A{twitter_handle})%20until%3A{to_date}%20since%3A{from_date}%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live'
    response = requests.get(url)
    driver = webdriver.Firefox()
    driver.get(url)
    time.sleep(10) 
    
    # scrolls down to the bottom of the page
    infinity_scroll(driver,filename)

    driver.quit()
    
    return

In [6]:
# twitter_handle = 'gsk'
# twitter_handle = 'sanofi'
# twitter handle = 'AstraZeneca'
twitter_handle = 'realdonaldtrump'

# twitter_handle = 'elonmusk'
from_date = '2020-01-01'
to_date = '2020-09-24'


scrape_tweets(twitter_handle,from_date,to_date,'trump_tweets_Jan_2020_Sept_2020.xml')

# Processing Tweet Data
The following code processes the data pulled from the XML file to create a listing of all Tweets.

In [28]:
def get_tweets2(filename):
    with open(filename,'r') as f:
        soupObj = BeautifulSoup(f,'html.parser')
    
    total_tweets = len(soupObj.findAll(class_="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-1mi0q7o"))
    tweet_listing = []
    time_listing = []
    
    tweet_block = soupObj.find(class_="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-1mi0q7o")
    
    for i in range(total_tweets):
        try:
            
            tweet = tweet_block.find('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
            tweet_listing.append(tweet.text)
        
        except AttributeError:
            tweet_listing.append(np.NaN)
        
        try:
            timestamp = tweet_block.find('time')
            time_listing.append((timestamp.find_next('time').attrs)['datetime'])
        
        except AttributeError:
            time_listing.append(np.NaN)

        tweet_block = tweet_block.find_next(class_="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-1mi0q7o")

    return pd.DataFrame({'Time':time_listing,'Tweet':tweet_listing})

In [49]:
# filepath = 'astrazeneca_tweets_Oct_2017_Sept_24_2020.xml'
# filepath = 'gsk_tweets_Oct_2017_Sept_24_2020.xml'
# filepath = 'pfizer_tweets_Oct_2017_Sept_24_2020.xml'
# filepath = 'elon_tweets_Sept_2019_Sept_2020.xml'
filepath = 'trump_tweets_Jan_2020_Sept_2020.xml'

df = get_tweets2(filepath)

In [60]:
df_copy = df.copy()

In [61]:
# MANUAL ADJUSTMENT
## Not sure why but all the tweets scraped are offset by 1 with the first date value missing.
## In the interest of time, I'm shifting the cells and manually adding it back:

df_copy['Time'] = df_copy[['Time']].shift()
df_copy.loc[[0,0],'Time'] = "2020-09-23T14:03:08.000Z"

In [64]:
df_copy.head()

,Time,Tweet
0,2020-09-23T14:03:08.000Z,"Very important that, in order to watch that AL..."
1,2020-09-23T13:58:47.000Z,Bay of Pigs Veterans & Hispanic Heritage Remar...
2,2020-09-23T13:21:10.000Z,White House News Conference today at 6:00 P.M....
3,2020-09-23T11:22:58.000Z,I hardly know Cindy McCain other than having p...
4,2020-09-22T22:53:22.000Z,"A few weeks ago, I BANNED efforts to indoctrin..."


In [66]:
# df.to_csv('gsk_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('pfizer_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('astrazeneca_tweets_Oct_2017_Sept_24_2020.csv')
# df.to_csv('elon_tweets_Sept_2019_Sept_2020.csv')

df_copy.to_csv('trump_tweets_Jan_2020_Sep_2020.csv')

---------------------------------------
# Code Graveyard
Code that is not currently in use.

In [7]:
# used to extract tweets from xml file, however it was found not to work in tweets that were censored
# replaced by get_tweets2
def get_tweets(filename):

    with open(filename,'r') as f:
        soupObj = BeautifulSoup(f,'html.parser')
           
    num_tweets = len(soupObj.find_all('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0"))
    num_times = len(soupObj.find_all('time'))
    
    timestamp = soupObj.find('time')
    tweet = soupObj.find('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
    
    tweet_listing = [tweet.text]
    time_listing = [timestamp.attrs['datetime']]
    
    for i in range(max(num_tweets,num_times)):
        try:
            #pull timestamps
            time_listing.append((timestamp.find_next('time').attrs)['datetime'])
            timestamp = timestamp.find_next('time')
            
        except AttributeError:
            time_listing.append(np.NaN)
        try:
            # pull tweets
            tweet_listing.append(tweet.find_next('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0").text)
            tweet = tweet.find_next('div',"css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
        
        except AttributeError:
            tweet_listing.append(np.NaN)


    return pd.DataFrame({'Time':time_listing,'Tweet':tweet_listing})



In [205]:
# GetOldTweets3 no longer works since twitter changed the endpoint. Only been an issue for 4 days, but may be fixed.
# save code incase the developers fix it.
# import GetOldTweets3 as got
# tweetCriteria = got.manager.TweetCriteria().setQuerySearch("realDonaldTrump").setMaxTweets(2)
# tweet = got.manager.TweetManager.getTweets(tweetCriteria)


In [ ]:
## Code used in testing
# variables for testing scrape_tweets()
# 'https://twitter.com/search?q=(from%3Arealdonaldtrump)%20until%3A2020-07-01%20since%3A2020-01-01%20-filter%3Alinks%20-filter%3Areplies&src=typed_query&f=live'
# twitter_handle = 'realdonaldtrump'
# from_date = '2020-01-01'
# to_date = '2020-07-01'

# other scrapers:
# snscrape's jsonl
# https://github.com/twintproject/twint/issues/918#issuecomment-696448934